In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop

In [ ]:
train = ImageDataGenerator(rescale=1/255)
validation = ImageDataGenerator(rescale=1/255)

In [ ]:
train_dataset = train.flow_from_directory("/content/drive/MyDrive/tf_workspace/training_cnn/images/training",target_size=(200,200),batch_size=8)
validation_dataset = train.flow_from_directory("/content/drive/MyDrive/tf_workspace/training_cnn/images/validation",target_size=(200,200),batch_size=8)

Found 4127 images belonging to 4 classes.
Found 486 images belonging to 4 classes.


In [ ]:
train_dataset.class_indices

{'bee': 0, 'bumbee': 1, 'insect': 2, 'vespula': 3}

In [ ]:
def convolutional_block(x, filter):
    # copy tensor to variable called x_skip
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same', strides = (2,2))(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    # Processing Residue with conv(1,1)
    x_skip = tf.keras.layers.Conv2D(filter, (1,1), strides = (2,2))(x_skip)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])     
    x = tf.keras.layers.Activation('relu')(x)
    return x
def identity_block(x, filter):
    # copy tensor to variable called x_skip
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])     
    x = tf.keras.layers.Activation('relu')(x)
    return x
def ResNet34(shape = (32, 32, 3), classes = 10):
    # Step 1 (Setup Input Layer)
    x_input = tf.keras.layers.Input(shape)
    x = tf.keras.layers.ZeroPadding2D((3, 3))(x_input)
    # Step 2 (Initial Conv layer along with maxPool)
    x = tf.keras.layers.Conv2D(64, kernel_size=7, strides=2, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same')(x)
    # Define size of sub-blocks and initial filter size
    block_layers = [3, 4, 6, 3]
    filter_size = 64
    # Step 3 Add the Resnet Blocks
    for i in range(4):
        if i == 0:
            # For sub-block 1 Residual/Convolutional block not needed
            for j in range(block_layers[i]):
                x = identity_block(x, filter_size)
        else:
            # One Residual/Convolutional Block followed by Identity blocks
            # The filter size will go on increasing by a factor of 2
            filter_size = filter_size*2
            x = convolutional_block(x, filter_size)
            for j in range(block_layers[i] - 1):
                x = identity_block(x, filter_size)
    # Step 4 End Dense Network
    x = tf.keras.layers.AveragePooling2D((2,2), padding = 'same')(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(512, activation = 'relu')(x)
    x = tf.keras.layers.Dense(classes, activation = 'softmax')(x)
    model = tf.keras.models.Model(inputs = x_input, outputs = x, name = "ResNet34")
    return model
def customModel():
  return tf.keras.models.Sequential([tf.keras.layers.Conv2D(32,(3,3),activation = "relu", input_shape =(200,200,3)),
                              #Preprocessing
                              tf.keras.layers.RandomContrast(factor=0.1, seed=None),
                              tf.keras.layers.RandomRotation(factor=1,fill_mode='reflect',interpolation='bilinear',seed=None,fill_value=0.0),
                              tf.keras.layers.RandomFlip(),
                              #Training
                              tf.keras.layers.MaxPool2D(3,3),
                              tf.keras.layers.Conv2D(32,(3,3),activation = "relu"),
                              tf.keras.layers.MaxPool2D(3,3),
                              tf.keras.layers.Conv2D(64,(3,3),activation = "relu"),
                              tf.keras.layers.MaxPool2D(3,3),
                              tf.keras.layers.Conv2D(128,(3,3),activation = "relu"),
                              tf.keras.layers.MaxPool2D(3,3),
                              tf.keras.layers.Flatten(),
                              tf.keras.layers.Dense(512, activation = "relu"),
                              tf.keras.layers.Dense(4, activation = "softmax"),
                              ])

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')
tf.device('/device:GPU:0')
model = tf.keras.models.Sequential([
    # Input layer
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3), padding='same'),
    
    # Preprocessing
    tf.keras.layers.RandomRotation(factor=1, fill_mode='reflect', interpolation='bilinear', seed=None, fill_value=0.0),
    tf.keras.layers.RandomFlip(),
    
    # Feature extraction
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    
    # Classifier
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(4, activation='softmax')
])
print(type(model))

<class 'keras.engine.sequential.Sequential'>


In [ ]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "/content/drive/MyDrive/tf_workspace/training_cnn_3/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
batch_size = 8
# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=100*3)

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))
model.compile(optimizer='adam', metrics=["accuracy"], loss=tf.keras.losses.CategoricalCrossentropy())

In [ ]:
with tf.device('/device:GPU:0'):
  model_fit = model.fit(train_dataset,steps_per_epoch = 3, batch_size = batch_size, callbacks=[cp_callback], epochs = 1000, validation_data=validation_dataset)

Epoch 1/1000


3/3 [==============================] - 10s 2s/step - loss: 42.9243 - accuracy: 0.3750 - val_loss: 1.6383 - val_accuracy: 0.2428
Epoch 2/1000
3/3 [==============================] - 2s 745ms/step - loss: 51.3119 - accuracy: 0.3333 - val_loss: 1.8784 - val_accuracy: 0.3621
Epoch 3/1000
3/3 [==============================] - 2s 754ms/step - loss: 31.6151 - accuracy: 0.5000 - val_loss: 16.1104 - val_accuracy: 0.2737
Epoch 4/1000
3/3 [==============================] - 2s 765ms/step - loss: 65.9794 - accuracy: 0.4583 - val_loss: 33.0895 - val_accuracy: 0.2695
Epoch 5/1000
3/3 [==============================] - 3s 1s/step - loss: 76.8141 - accuracy: 0.2917 - val_loss: 25.6336 - val_accuracy: 0.2881
Epoch 6/1000
3/3 [==============================] - 2s 734ms/step - loss: 56.6343 - accuracy: 0.2500 - val_loss: 32.6966 - val_accuracy: 0.1646
Epoch 7/1000
3/3 [==============================] - 2s 746ms/step - loss: 46.4196 - accuracy: 0.5000 - val_loss: 35.2970 - val_accuracy: 0.1605
Epoch 8/1000

In [ ]:
validation_dataset.class_indices

NameError: ignored

In [ ]:
from tensorflow.keras.preprocessing import image
validation_dataset.class_indices
dir_path = "/content/drive/MyDrive/tf_workspace/training_cnn/images/training"
detWrong =[]
classes = 4
detWrongElement = []
for count in range(classes):
  detWrongElement.append([0,0,0,0,0])
for i in os.listdir(dir_path):
  for files in os.listdir(os.path.join(dir_path, i)):
    active_dir = os.path.join(dir_path, i)
    imagePath = os.path.join(active_dir, files)
    img = image.load_img(imagePath, target_size=(200,200))
    X = image.img_to_array(img)
    X = np.expand_dims(X,axis=0)
    images = np.vstack([X])
    val = model.predict(images)
    detectionList = val.tolist()
    filename, _ = os.path.splitext(files)
    fileType = filename.split('(')
    fileTypeValue = None
    if(fileType[0] == "bee"):
      fileTypeValue = 0
    if(fileType[0] == "bumbeBee" or fileType[0] == "bumbleBee"):
      fileTypeValue = 1
    if(fileType[0] == "insect"):
      fileTypeValue = 2
    if(fileType[0] == "vespula"):
      fileTypeValue = 3
    if(fileTypeValue != None and detectionList[0][fileTypeValue] < 0.8):
      detWrong.append([detectionList[0],files])
      detWrongElement[fileTypeValue][0] +=1
      for index, element in enumerate(detectionList[0]):
        if(element >= 0.8):
          detWrongElement[fileTypeValue][index+1] +=1
    elif(fileTypeValue == None):
      print(filename)
print("bee, bumblebee, insect, vespula")
print(detWrongElement)

NameError: ignored

### Get number of files within the folder

In [ ]:
dirPath = "/content/drive/MyDrive/tf_workspace/training_cnn/images/training"
os.listdir(dirPath)
for elements in os.listdir(dirPath):
  print(f"{elements}: {len(os.listdir(os.path.join(dirPath, elements)))}")

vespula: 273
bumbee: 774
bee: 1068
insect: 2012
